图片处理函数

In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

特别注意cv2.rectangle会在你个的那张图上画，意思就是会改变原图，改变了原来的after，而且还传播到了函数外面，会出现很多框框

In [16]:
def detect(former,after):
    img_former = np.copy(former)
    img_after = np.copy(after)
    # img_former = former
    # img_after = after
    #先转为灰度图
    img_former_gray = cv2.cvtColor(img_former, cv2.COLOR_BGR2GRAY)
    img_after_gray = cv2.cvtColor(img_after, cv2.COLOR_BGR2GRAY)
    new_image = cv2.absdiff(img_former_gray, img_after_gray)
    
    ret,thresh = cv2.threshold(new_image,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    kernel = np.ones((30,30), np.uint8)
    img_dilate = cv2.dilate(thresh, kernel, iterations= 2)  
    # img_erode = cv2.erode(img_dilate, kernel, iterations=2)
    contours, hierarchy = cv2.findContours(img_dilate,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    # img=cv2.drawContours(img_after,contours,-1,(0,255,0),5)
    # #链码：
    # temp = []
    # for i in range(len(contours)):
    #     for j in range(len(contours[i])):
    #         a = contours[i][j][0][0] - contours[i][j-1][0][0]
    #         b = contours[i][j][0][1] - contours[i][j-1][0][1]
    #         print(a,b)
    #         if(a==1 and b==0):
    #             temp.append(0)
    #         elif(a==1 and b==1):
    #             temp.append(1)
    #         elif(a==0 and b==1):
    #             temp.append(2)
    #         elif(a==-1 and b==1):
    #             temp.append(3)
    #         elif(a==-1 and b==0):
    #             temp.append(4)
    #         elif(a==-1 and b==-1):
    #             temp.append(5)
    #         elif(a==0 and b==-1):
    #             temp.append(6)
    #         elif(a==1 and b==-1):
    #             temp.append(7)
    # if(len(temp)>0):
    #     print(temp)
    
    rects = []
    for i in range(len(contours)):
        temp = cv2.boundingRect(contours[i])
        rects.append(temp)
        
    center_x = 0
    center_y = 0
    correct_x =0
    correct_y =0
    maxw =0
    maxh =0
    for i in range(len(rects)):
        [x , y, w, h] = rects[i]
        if(w*h>maxw*maxh):
            maxw = w
            maxh = h
            correct_x = x
            correct_y = y
    cv2.rectangle(img_after, (correct_x, correct_y), (correct_x + maxw, correct_y + maxh), (0, 255, 0), 2)    
        # center_x = center_x + x + w/2
        # center_y = center_y + y + h/2
        # num=num+1
    # if(num == 0):
    #     center_x = 0
    #     center_y = 0
    # else:
    #     center_x =center_x/num
    #     center_y =center_y/num
    if (len(rects)>0):
        center_x = correct_x + maxw/2
        center_y = correct_y + maxh/2
    else:
        center_x = 0
        center_y = 0
    return center_x, center_y , img_after 


In [17]:
frame_former = ""
beijing = ""
videpath= './people.mp4'
capture = cv2.VideoCapture(videpath)
frameCount = capture.get(cv2.CAP_PROP_FRAME_COUNT); 
fps = capture.get(cv2.CAP_PROP_FPS)
x=0
y=0
centers_x = []
centers_y = []
for i in range(int(frameCount)):
    ret, frame = capture.read() 
    if ret:
        cv2.imshow("frame", frame);
        if (i == 0):
            x,y,result = detect(frame,frame)
            beijing = np.copy(frame)
        else:
            x,y,result = detect(beijing,frame)
        if(len(centers_x)>=25):
            centers_x.pop()
            centers_y.pop()
        centers_x.insert(0,x)
        centers_y.insert(0,y)
        for p in range(len(centers_x)):
             cv2.circle(result, (int(centers_x[p]), int(centers_y[p])), 3, (0,0,255),1)
        cv2.imshow("result", result);
        key = cv2.waitKey(int(1000/fps))
        if ((key) == 27):
            capture.release() 
            break
        elif(key==ord('q')):
            print(i)
            cv2.waitKey(0)
        frame_former = frame
    else:
        capture.release()
capture.release()
cv2.destroyAllWindows() 

In [4]:
i = 0
result = 0
frame_former = 0
videpath= './people.mp4'
capture = cv2.VideoCapture(videpath)
fps = capture.get(cv2.CAP_PROP_FPS)
while capture.isOpened(): 
    i = i + 1
    ret, frame = capture.read() 
    if ret:
        cv2.imshow('frame', frame) 
        cv2.waitKey(int(1000/fps))
        if  i == 1:
            result = detect(frame,frame)
        else:
            result = detect(frame_former,frame)
        frame_former = np.copy(frame)
        cv2.imshow("result",result)
        key = cv2.waitKey(int(1000/fps))
        if key == ord('q'):         # 若是键盘输入'q',则退出，释放视频
            capture.release()           # 释放视频
            break
    else:
        capture.release()
cv2.destroyAllWindows() 
        

In [ ]:
cap = cv2.VideoCapture('2.mp4')     # 读取视频
while cap.isOpened():               # 当视频被打开时：
    ret, frame = cap.read()         # 读取视频，读取到的某一帧存储到frame，若是读取成功，ret为True，反之为False
    if ret:                         # 若是读取成功
        cv2.imshow('frame', frame)  # 显示读取到的这一帧画面
        key = cv2.waitKey(25)       # 等待一段时间，并且检测键盘输入
        if key == ord('q'):         # 若是键盘输入'q',则退出，释放视频
            cap.release()           # 释放视频
            break
    else:
        cap.release()
cv2.destroyAllWindows()             # 关闭所有窗口
 